In [1]:
from google.cloud import bigquery

https://cloud.google.com/docs/authentication/getting-started

SQL Kaggle-4668a0b2f445.json access to cloud resources

export GOOGLE_APPLICATION_CREDENTIALS="[PATH]"


In [2]:
import os

In [3]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/pavannaik/Desktop/API/sql-kaggle-244915-e2d3c0bfe87b.json"

In [4]:
client = bigquery.Client()

#### Construct a reference to the "hacker_news" dataset

In [5]:
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

#### API request - fetch the dataset

In [6]:
dataset = client.get_dataset(dataset_ref)

#### List all the tables in the "hacker_news" dataset

In [7]:
# list_tables() method to list the tables in the dataset.
tables = list(client.list_tables(dataset))

In [8]:
for table in tables:
    print(table.table_id)

comments
full
full_201510
stories


#### Construct a refernece to the full table

In [9]:
table_ref = dataset_ref.table("full")

#### API reference to fetch the table

In [10]:
table = client.get_table(table_ref)

In [11]:
# Print information on all the columns in the "full" table in the "hacker_news" dataset
table.schema

[SchemaField('by', 'STRING', 'NULLABLE', "The username of the item's author.", ()),
 SchemaField('score', 'INTEGER', 'NULLABLE', 'Story score', ()),
 SchemaField('time', 'INTEGER', 'NULLABLE', 'Unix time', ()),
 SchemaField('timestamp', 'TIMESTAMP', 'NULLABLE', 'Timestamp for the unix time', ()),
 SchemaField('title', 'STRING', 'NULLABLE', 'Story title', ()),
 SchemaField('type', 'STRING', 'NULLABLE', 'Type of details (comment, comment_ranking, poll, story, job, pollopt)', ()),
 SchemaField('url', 'STRING', 'NULLABLE', 'Story url', ()),
 SchemaField('text', 'STRING', 'NULLABLE', 'Story or comment text', ()),
 SchemaField('parent', 'INTEGER', 'NULLABLE', 'Parent comment ID', ()),
 SchemaField('deleted', 'BOOLEAN', 'NULLABLE', 'Is deleted?', ()),
 SchemaField('dead', 'BOOLEAN', 'NULLABLE', 'Is dead?', ()),
 SchemaField('descendants', 'INTEGER', 'NULLABLE', 'Number of story or poll descendants', ()),
 SchemaField('id', 'INTEGER', 'NULLABLE', "The item's unique id.", ()),
 SchemaField('ran

In [12]:
# Preview the first five lines of the "full" table
client.list_rows(table, max_results=5).to_dataframe()

,by,score,time,timestamp,title,type,url,text,parent,deleted,dead,descendants,id,ranking
0,danielam,NaN,1501190219,2017-07-27 21:16:59+00:00,,comment,,"If anything, I think you corroborate the artic...",14860585.0,None,None,NaN,14869651,None
1,gazrogers,NaN,1329135363,2012-02-13 12:16:03+00:00,,comment,,"Looks ok for me - Chrome 17.0.963.46 m, Window...",3585273.0,None,None,NaN,3585285,None
2,megamark16,NaN,1470927294,2016-08-11 14:54:54+00:00,,comment,,I think part of what I like about this idea is...,12267908.0,None,None,NaN,12268682,None
3,masukomi,NaN,1454706671,2016-02-05 21:11:11+00:00,,comment,,does this just NOT work for private repos?,11042366.0,None,None,NaN,11044653,None
4,aaronbrethorst,106.0,1443668243,2015-10-01 02:57:23+00:00,Secret developers of the video game industry,story,http://www.polygon.com/2015/9/30/9394355/the-s...,,NaN,None,None,36.0,10308958,None


In [13]:
# list 10 elements from 1st column
client.list_rows(table, selected_fields=table.schema[:1], max_results=10).to_dataframe()

,by
0,danielam
1,gazrogers
2,megamark16
3,masukomi
4,aaronbrethorst
5,opportune
6,FractalNerve
7,sporkologist
8,TrevorJ
9,AlisdairO


In [14]:
query_popular = """
                SELECT parent, COUNT(id)
                FROM `bigquery-public-data.hacker_news.comments`
                GROUP BY parent
                HAVING COUNT(id) > 10
                """

In [15]:
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)

In [16]:
query_job = client.query(query_popular, job_config=safe_config)

In [17]:
popular_comments = query_job.to_dataframe()

In [18]:
popular_comments.head()

,parent,f0_
0,3873271,50
1,5308611,63
2,5176140,63
3,9269660,51
4,9439286,69


In [19]:
query_improved = """
                 SELECT parent, COUNT(1) AS NumPosts
                 FROM `bigquery-public-data.hacker_news.comments`
                 GROUP BY parent
                 HAVING COUNT(1) > 10
                 """

In [20]:
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job1 = client.query(query_improved, job_config=safe_config)

In [21]:
improved_df = query_job1.to_dataframe()

In [22]:
improved_df.head()

,parent,NumPosts
0,2214158,42
1,9211214,48
2,6821105,38
3,9186013,42
4,157443,41


In [23]:
# Query to select prolific commenters and post counts
prolific_commenters_query = """
                            SELECT author, COUNT(1) AS NumPosts
                            FROM `bigquery-public-data.hacker_news.comments`
                            GROUP BY author
                            HAVING COUNT(1) > 10000""" # Your code goes here


safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**9)
query_job = client.query(prolific_commenters_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
prolific_commenters = query_job.to_dataframe()

# View top few rows of results
print(prolific_commenters.head())

         author  NumPosts
0           eru     10448
1       rbanffy     10557
2  dragonwriter     10723
3          None    227736
4         DanBC     12902


In [24]:
#How many comments have been deleted? (If a comment was deleted, the `deleted` column in the 
#comments table will have the value `True`.)

deleted_query = """
                SELECT COUNT(1) AS NumDeletedPosts
                FROM `bigquery-public-data.hacker_news.comments`
                WHERE deleted = TRUE
                """

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**9)
query_job_del = client.query(deleted_query, job_config = safe_config)

num_deleted_posts = query_job_del.to_dataframe()

print(num_deleted_posts.head())

   NumDeletedPosts
0           227736
